In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("jr58-6ysp", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


,usa_or_hhsregion,week_ending,variant,share,share_hi,share_lo,count_lt10,modeltype,time_interval,creation_date
0,1,2023-04-22T00:00:00.000,XBB.1.9.1,0.06049646809697151,0.12164641916751862,0.027775244787335396,NULL,smoothed,weekly,2023-05-05T00:00:00.000
1,1,2023-04-22T00:00:00.000,BQ.1.1,0.001003597630187869,0.0016649839235469699,0.0005953878862783313,NULL,smoothed,weekly,2023-05-05T00:00:00.000
2,1,2023-04-22T00:00:00.000,XBB.1.5.1,0.029900113120675087,0.03945222496986389,0.022545939311385155,NULL,smoothed,weekly,2023-05-05T00:00:00.000
3,1,2023-04-22T00:00:00.000,FD.2,0.0014671562239527702,0.0045453826896846294,0.0003943904011975974,NULL,smoothed,weekly,2023-05-05T00:00:00.000
4,1,2023-04-22T00:00:00.000,XBB.1.9.2,0.008673355914652348,0.02374914474785328,0.0027599986642599106,NULL,smoothed,weekly,2023-05-05T00:00:00.000
